### Deploys Triton custom models from [azureml-examples](https://github.com/azure/azureml-examples) repo

In [1]:
#### clone the github repo
!git clone https://github.com/azure/azureml-examples

fatal: destination path 'azureml-examples' already exists and is not an empty directory.


In [3]:
!ls -R azureml-examples/cli/endpoints/online/triton/single-model/models

model_1

azureml-examples/cli/endpoints/online/triton/single-model/models/model_1:
1

azureml-examples/cli/endpoints/online/triton/single-model/models/model_1/1:
model.onnx


In [6]:
%%writefile resources/triton_custom_model.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: triton-custom
  namespace: seldon
spec:
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: https://github.com/Azure/azureml-examples/tree/main/cli/endpoints/online/triton/single-model/models/model_1/1
      name: triton-custom
      type: MODEL
    name: default
    replicas: 1
  protocol: v2

Overwriting resources/triton_custom_model.yaml


In [8]:
#!kubectl delete -f resources/triton_custom_model.yaml
!kubectl apply -f resources/triton_custom_model.yaml

seldondeployment.machinelearning.seldon.io "triton-custom" deleted


In [ ]:
!kubectl rollout status -n seldon deploy/$(kubectl get deploy -l seldon-deployment-id=triton-custom -n seldon -o jsonpath='{.items[0].metadata.name}')